In [1]:
import pandas as pd

In [5]:
inactive_strategies_df = {'Date':['YYYY-MM-DD'], 'Strategy Name':[[0]]}
pd.DataFrame(inactive_strategies_df).set_index('Date').to_csv('inactive_strategies.csv')

In [6]:
df = pd.read_csv('inactive_strategies.csv')

In [8]:
df.set_index('Date', inplace=True)

In [10]:
df.loc['YYYY-MM-D1'] = [0]

In [12]:
df.to_csv('inactive_strategies.csv')